<a href="https://colab.research.google.com/github/metaabhinav/fine-tuning-code-anyscale/blob/master/fine_tuning_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   SUBSTITUTION_MODEL:

  1 - mistralai/Mistral-7B-Instruct-v0.1:120_golden_dataset2_substitution_only_try2:wRlCPqb
    
  2 - meta-llama/Llama-2-70b-chat-hf:120_golden_dataset2_substitution_only_try2:Klj2j62

  3- meta-llama/Llama-2-70b-chat-hf:145_golden_dataset2_substitution_only:3Sq7lLG

2.   REDACTION_MODEL :   

  1 - mistralai/Mistral-7B-Instruct-v0.1:120_golden_dataset2_redaction_only_try2:aRbZbUc
  
  2- meta-llama/Llama-2-70b-chat-hf:120_golden_dataset2_redaction_only_try2:zOZ0nWe

  3- meta-llama/Llama-2-70b-chat-hf:145_golden_dataset2_redaction_only:UiYLtjB



Set fine tune model and user input

In [ ]:
substitution_model = "meta-llama/Llama-2-70b-chat-hf:145_golden_dataset2_substitution_only:3Sq7lLG"
redaction_model = "meta-llama/Llama-2-70b-chat-hf:145_golden_dataset2_redaction_only:UiYLtjB"
user_input = "Gender of Apple CEO is male"

Substitution process


In [ ]:
import time
start_time_substitution = time.time()

import os
import requests

session_request = requests.Session()
API_BASE="https://api.endpoints.anyscale.com/v1"
TOKEN = "esecret_sipwvajkxez1pirq7crckbe3n1"
system_content =" You are to take the user input and substitute any company name, or specific brand name, trademark name or project name, substituting it with a descriptive and narrow category of the company, brand, or domain of the project, respectively"
api_base = API_BASE
token = TOKEN
url = f"{api_base}/chat/completions"
body = {
  "model": f"{substitution_model}",
  "messages": [{"role": "system", "content": f"{system_content}"}, {"role": "user", "content": f"{user_input}"}],
  "temperature": 0.1
}
# print(url)
# print(token)
# print(body)
# print(session_request)
with session_request.post(url, headers={"Authorization": f"Bearer {token}"}, json=body) as resp:
    resp_out = resp.json()
    print(resp.json())

end_time_substitution = time.time()

execution_time_substitution = end_time_substitution - start_time_substitution
print(f"Execution time: {execution_time_substitution} seconds")

{'id': 'meta-llama/Llama-2-70b-chat-hf:145_golden_dataset2_substitution_only:3Sq7lLG-tFjHVPfPj_Utmxeom92nGbkcJYz_AMfJw_I4pz1sPEs', 'object': 'text_completion', 'created': 1706075948, 'model': 'meta-llama/Llama-2-70b-chat-hf:145_golden_dataset2_substitution_only:3Sq7lLG', 'choices': [{'message': {'role': 'assistant', 'content': ' Substitution Output: {\'Gender of [Tech Company] CEO is male\'}, SUBSTITUTION_MAP: {"[Tech Company]": "Apple"}  ', 'tool_calls': None, 'tool_call_id': None}, 'index': 0, 'finish_reason': 'stop', 'logprobs': None}], 'usage': {'prompt_tokens': 79, 'completion_tokens': 40, 'total_tokens': 119}}
Execution time: 2.6203408241271973 seconds


In [ ]:
import json
SUB_MAP = json.loads(resp.json()['choices'][0]['message']['content'].split("SUBSTITUTION_MAP")[1][2:])
print(SUB_MAP)
input_redaction = resp.json()['choices'][0]['message']['content'].split("SUBSTITUTION_MAP")[0][:-4].split("Substitution Output:")[1][3:]
print(input_redaction)


{'[Tech Company]': 'Apple'}
Gender of [Tech Company] CEO is male


Redaction process


In [ ]:
start_time_redaction = time.time()
session_request = requests.Session()
API_BASE="https://api.endpoints.anyscale.com/v1"
TOKEN = "esecret_sipwvajkxez1pirq7crckbe3n1"
system_content ="You are to take the user input and redact any sensitive enterprise-specific content from it. Redact any company proprietary information from the input statement using the following rules, and provide the rephrased statement and its corresponding REDACTION_MAP. Do NOT try to answer the query itself. Rule# 1: Redact all Personally Identifiable Information (PII) of any identified entities in user input, such as a person name, address, Credit-card number, any secret, etc, replacing it with generic entity tokens like [Redacted Person1], [Redacted Person2], etc for person name; [Redacted Address1], etc for an address, [Redacted CC1], etc for a Credit-card, etc, etc;  Rule# 2: Replace any specifics related to the USER's employer with generics as much as possible such as: (a) Redacting any specific numbers related to USER's employer, like revenue or market share numbers, percentages, etc, replacing them with [X1], [X2], [Y1], [Y2], [Z1]%, etc, (b) Redacting launch dates or announcement dates of any new products or projects from USER's employer with [Redacted Date1], etc. Rule# 3: Redact any source code snippets present in the user input. Rule# 4: Remove any profanity that is present in the input statements"
# system_content="You are to take the user input and redact/replace any confidential information belonging to the user's employer from it, including (1) all Personally Identifiable Information (PII), such as a person name, address, Credit-card number, any secret, etc, replacing it with generic entity tokens like [Redacted Person1], [Redacted Person2], etc for person name; [Redacted Address1], etc for an address, [Redacted CC1], etc for a Credit-card, etc, etc; (2) any specific numbers or dates, like revenue or market share numbers, percentages, etc, replacing them with [X1], [X2], [Y1], [Y2], [Z1]%, etc,  launch dates or announcement dates of any new products or projects replacing them with [Redacted Date1], etc. (3): Redact any source code snippets present in the user input, (4): Remove any profanity that is present in the input statements."
user_input = input_redaction
api_base = API_BASE
token = TOKEN
url = f"{api_base}/chat/completions"
body2 = {
  "model": f"{redaction_model}",
  "messages": [{"role": "system", "content": f"{system_content}"}, {"role": "user", "content": f"{user_input}"}],
  "temperature": 0
}
# print(url)
# print(token)
# print(body)
# print(session_request)
with session_request.post(url, headers={"Authorization": f"Bearer {token}"}, json=body2) as response:
    resp_out = response.json()
    print(response.json())

end_time_redaction = time.time()

execution_time_redaction = end_time_redaction - start_time_redaction
print(f"Execution time: {execution_time_redaction} seconds")

{'id': 'meta-llama/Llama-2-70b-chat-hf:145_golden_dataset2_redaction_only:UiYLtjB-I_5OyNMhn3RiIS7B78F3NtIBTOpGWsXywrpHDubtkM0', 'object': 'text_completion', 'created': 1706075954, 'model': 'meta-llama/Llama-2-70b-chat-hf:145_golden_dataset2_redaction_only:UiYLtjB', 'choices': [{'message': {'role': 'assistant', 'content': ' Redaction Output: {\'Gender of [Tech Company] CEO is [Redacted Person1]\'}, REDACTION_MAP: {"[Redacted Person1]": "Sundar Pichai"}  ', 'tool_calls': None, 'tool_call_id': None}, 'index': 0, 'finish_reason': 'stop', 'logprobs': None}], 'usage': {'prompt_tokens': 351, 'completion_tokens': 49, 'total_tokens': 400}}
Execution time: 5.549074649810791 seconds


In [ ]:
print("SUBSTITUTION_MAP: " , SUB_MAP)
RED_MAP = json.loads(response.json()['choices'][0]['message']['content'].split("REDACTION_MAP")[1][2:])
print("REDACTION_MAP: ", RED_MAP)
output = response.json()['choices'][0]['message']['content'].split("REDACTION_MAP")[0][:-4].split("Redaction Output:")[1][3:]
print("FINAL_OUTPUT:" , output)
print("TOTAL_TIME: " , execution_time_substitution + execution_time_redaction)

SUBSTITUTION_MAP:  {'[Tech Company]': 'Apple'}
REDACTION_MAP:  {'[Redacted Person1]': 'Sundar Pichai'}
FINAL_OUTPUT: Gender of [Tech Company] CEO is [Redacted Person1]
TOTAL_TIME:  8.169415473937988
